# Fine-tuning a Machine Translation Transformer Model

The objective of the project is to fine-tune a pretrained Helsinki-NLP/opus-mt-en-de Transformer model with the Europarl English-German parallel corpus.
See the links to datasets and tutorials followed below. 

## Fine-tuning

Importing libraries:

In [ ]:
import numpy as np
import evaluate
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset

Defining English as a source language and German as a target language:

In [ ]:
SRC = "en"
TG = "de"

In [ ]:
parallel_data = load_dataset("csv", data_files= "de-en/train_set.csv")
parallel_data = parallel_data["train"].train_test_split(test_size=0.15)

Dataset preprocessing (tokenization) in order to use the produced tokenized sentences as inputs for the transformer model:

In [ ]:
def preprocess_function(examples):
    inputs = [example for example in examples[SRC]]
    targets = [example for example in examples[TG]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
tokenized_inputs = parallel_data.map(preprocess_function, batched=True)

Loading model and creating a data collator:

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Functions for model evaluation with SacreBLEU metrics:

In [ ]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Writing training arguments:

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="europarl-de-en",
    evaluation_strategy="epoch",
    learning_rate=0.00002,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=False,
)

#If the model is meant to be uploaded, set "push_to_hub" to True.

Creating a trainer object:

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_inputs["train"],
    eval_dataset=tokenized_inputs["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The model was pushed to the hub with the following code: trainer.push_to_hub()

Let's test the model.
Downloading the fine-tuned model from the hub:

In [ ]:
europarl_model = AutoModelForSeq2SeqLM.from_pretrained("marsim0/europarl-de-en")

Reading 20 a few new English sentences:

In [ ]:
with open("de-en/test.en", "r", encoding="windows-1252") as file:
    eng_sents = file.readlines()[:20]

Generating a dictionary with translations:

In [ ]:
def translate(text):
    tokenizer = AutoTokenizer.from_pretrained("marsim0/europarl-de-en")
    translations = {}
    for sent in text:
        inputs = tokenizer(sent, return_tensors="pt").input_ids
        outputs = europarl_model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations[sent] = decoded
    return translations

transdict = translate(eng_sents)

Run to see the output of the model:

In [ ]:
for sent in transdict:
    print(f"EN: {sent}DE: {transdict[sent]}\n")

You can also see the previously generated output of this model in the "europarl-de-en_output" file.

## Comparison

The comparison of both models demonstrates that the fine tuned model had a slightly better performance, even though fine tuning was performed on a small dataset.

| Model | Evaluation Loss | Sacrebleu Score |
| --- | --- | --- |
| Original | 1.506978 | 23.8212 |
| Fine Tuned | 1.424564 | 24.8548 |

## Sources

### Hugging Face Tutorials

[Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training)

[Translation](https://huggingface.co/docs/transformers/tasks/translation)

### Dataset

[Europarl](https://statmt.org/europarl/) English-German Parallel corpus from (taken from the [NAACL 2006 WORKSHOP](https://statmt.org/wmt06/shared-task/))